In [1]:
#bring in json file to dataframe

import pandas as pd
import json
from pandas.io.json import json_normalize

file_path = 'filepath_to_your_json'

with open(file_path, 'r') as f:
    data = json.load(f)
    
df = json_normalize(data,'locations')

In [88]:
#drop records with velocity, replace NaN with 0.0

df['velocity'].fillna(0.0,inplace = True)
df = df[df['velocity']<=0]
df.reset_index(inplace=True, drop = True)

In [2]:
#cleanup latitude and longitude

df['latitudeE7'] = df['latitudeE7'].fillna(0.0).astype(float)
df['longitudeE7'] = df['longitudeE7'].fillna(0.0).astype(float)

df['latitude'] = df.apply(lambda x: x['latitudeE7']/10000000.0,axis = 1)
df['longitude'] = df.apply(lambda x: x['longitudeE7']/10000000.0, axis = 1)

del df['longitudeE7']
del df['latitudeE7']

In [5]:
import gmplot

def plot_map(df, map_name):
    gmap = gmplot.GoogleMapPlotter(41.766834, -72.688671, 16)

    gmap.scatter(df.short_lat.tolist(), df.short_long.tolist(), '#FF6666', edge_width=10)
    gmap.heatmap(df.short_lat.tolist(), df.short_long.tolist())

    gmap.draw(map_name +"_map.html")

In [8]:
#best hartford heatmap

df2 = pd.DataFrame()
df2['latitude'] = df['latitude']
df2['longitude'] = df['longitude']
   
df2['short_lat'] = df2.apply(lambda x: ((x['latitude']).round(decimals = 4)), axis = 1)
df2['short_long'] = df2.apply(lambda x: ((x['longitude']).round(decimals = 4)), axis = 1)

df_hartford = df2.groupby(['short_lat','short_long']).agg({'latitude':'count'}).reset_index()

del df_hartford['latitude']

df_hartford.query('short_long < -72.584551', inplace = True)
df_hartford.query('short_long > -72.800233', inplace = True)
df_hartford.query('short_lat > 41.675283', inplace = True)
df_hartford.query('short_lat < 41.834845', inplace = True)

plot_map(df_hartford, 'hartford')

In [9]:
#best CT heatmap

df2 = pd.DataFrame()
df2['latitude'] = df['latitude']
df2['longitude'] = df['longitude']
   
df2['short_lat'] = df2.apply(lambda x: ((x['latitude']).round(decimals = 3)), axis = 1)
df2['short_long'] = df2.apply(lambda x: ((x['longitude']).round(decimals = 3)), axis = 1)

df_CT = df2.groupby(['short_lat','short_long']).agg({'latitude':'count'}).reset_index()

del df_CT['latitude']

df_CT.query('short_long < -71.797748', inplace = True)
df_CT.query('short_long > -73.653025', inplace = True)
df_CT.query('short_lat > 41.003823', inplace = True)
df_CT.query('short_lat < 42.033488', inplace = True)

plot_map(df_CT, 'CT')


(42.033488, -71.797748)

In [10]:
#best new England heatmap

df2 = pd.DataFrame()
df2['latitude'] = df['latitude']
df2['longitude'] = df['longitude']
   
df2['short_lat'] = df2.apply(lambda x: ((x['latitude']).round(decimals = 3)), axis = 1)
df2['short_long'] = df2.apply(lambda x: ((x['longitude']).round(decimals = 3)), axis = 1)

df_CT = df2.groupby(['short_lat','short_long']).agg({'latitude':'count'}).reset_index()


del df_CT['latitude']

df_CT.query('short_long < -70.762119', inplace = True)
df_CT.query('short_long > -75.881748', inplace = True)
df_CT.query('short_lat > 40.497342', inplace = True)
df_CT.query('short_lat < 43.293439', inplace = True)

plot_map(df_CT, 'New England')



In [25]:
import pandas as pd
import json
import os

#fill out your own info here
#the resolution took a lot of manipulation to get right,
#you may have to play with it depending on how your data works out
map_name = 'your_map_name_here'
map_center_lat = 41.766834
map_center_long = -72.688671
cord_low_lat = 40.497342
cord_high_lat = 43.293439
cord_low_long = -75.881748
cord_high_long = -70.762119
google_json = 'your_location_history_json_here'
coordinate_resolution = 3 

#plot HTML map
def plot_map(plot_df, map_name):
    import gmplot
    gmap = gmplot.GoogleMapPlotter(map_center_lat, map_center_long, 8)

    gmap.scatter(plot_df.short_lat.tolist(), plot_df.short_long.tolist(), '#FF6666', edge_width=10)
    gmap.heatmap(plot_df.short_lat.tolist(), plot_df.short_long.tolist())

    gmap.draw(map_name +"_map.html")

#open html map
def open_html_map(html_path):
    import webbrowser
    chrome_path = 'open -a /Applications/Google\ Chrome.app %s'
    webbrowser.get(chrome_path).open(html_path)
    
#bring in json file to dataframe
with open(google_json, 'r') as f:
    data = json.load(f)
    
df = json_normalize(data,'locations')

#cleanup latitude and longitude
df['latitudeE7'] = df['latitudeE7'].fillna(0.0).astype(float)
df['longitudeE7'] = df['longitudeE7'].fillna(0.0).astype(float)

df['latitude'] = df.apply(lambda x: x['latitudeE7']/10000000.0,axis = 1)
df['longitude'] = df.apply(lambda x: x['longitudeE7']/10000000.0, axis = 1)

del df['longitudeE7']
del df['latitudeE7']


#rounding the coordinates - resolution is important!
df2 = pd.DataFrame()
df2['latitude'] = df['latitude']
df2['longitude'] = df['longitude']
   
df2['short_lat'] = df2.apply(lambda x: ((x['latitude']).round(decimals = coordinate_resolution)), axis = 1)
df2['short_long'] = df2.apply(lambda x: ((x['longitude']).round(decimals = coordinate_resolution)), axis = 1)

#grouping by the resolution
#by doing this, we ignore how many times a place was visited
#but variation in GPS coordinates allows us to get away with
#it to make the map look good
df2 = df2.groupby(['short_lat','short_long']).agg({'latitude':'count'}).reset_index()
del df2['latitude']

df2.query('short_lat > ' + str(cord_low_lat), inplace = True)
df2.query('short_lat < ' + str(cord_high_lat), inplace = True)
df2.query('short_long > ' + str(cord_low_long), inplace = True)
df2.query('short_long < ' + str(cord_high_long), inplace = True)

plot_map(df2, map_name)
open_html_map(os.getcwd() + '/' + map_name +"_map.html")


In [11]:
def open_html_map(html_path):
    import webbrowser
    chrome_path = 'open -a /Applications/Google\ Chrome.app %s'
    webbrowser.get(chrome_path).open(html_path)
    
open_html_map('http://docs.python.org/')